In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.llms import Ollama

# 1 - Uma pipeline simples para retornar piadas em PT-BR

In [61]:
prompt = ChatPromptTemplate.from_messages([
    ('system', 'Você é um humorista e faz piadas conforme solicitado pelo usuário.'),
    ('human', 'Faça uma piada sobre o tema: {tema}')
])
model = Ollama(model='llama3.1')
parser = StrOutputParser()

chain = prompt | model | parser

chain.input_schema.schema()

{'properties': {'tema': {'title': 'Tema', 'type': 'string'}},
 'required': ['tema'],
 'title': 'PromptInput',
 'type': 'object'}

In [62]:
chain.invoke({'tema': 'cadeira'})

'Por aqui!\n\nVocê sabe por que a cadeira foi para o psicólogo? Porque estava sentindo-se "deslocada" e tinha problemas de "estar assentando as coisas"... E agora está "sentindo-se" muito melhor! (risos)'

In [63]:
stream = chain.stream({'tema': 'cadeira'})

for chunk in stream:
    print(chunk, end="", flush=True)
print()

Então, você sabe por que eu sempre sento na cadeira? É porque ela sempre tem um bom lugar para se sentar... E é só uma brincadeira! Mas sério, a minha casa está cheia de cadeiras em espera. Elas estão sempre dizendo: "Quando vou ser sentada?"


# 2 - Pipeline simples para fazer piadas em inglês

In [67]:
prompt = ChatPromptTemplate([
    ('system', 'You are a comedian and make jokes as requested by the user.'),
    ('human', 'Tell me a joke about the topic: {topic}')
])

model = Ollama(model='llama3.1')

parser = StrOutputParser()

chain = prompt | model | parser

print(chain.input_schema.schema())

# topic = input('Enter a topic for the question: ')

stream = chain.stream({'topic': 'chair'})
for chunk in stream:
    print(chunk, end="", flush=True)
print()

{'properties': {'topic': {'title': 'Topic', 'type': 'string'}}, 'required': ['topic'], 'title': 'PromptInput', 'type': 'object'}
Here's one:

"You know what they say, 'A good chair can be a real seat-saver.' But let's be honest, most chairs are just a bunch of legs... I mean, literally. They've got legs, an arse, and a back... that's basically just a fancy way of saying they're like your aunt after a few too many drinks!"


# 3 - Criando pipeline de tradução para PT-BR

In [42]:
from langchain_core.prompts import PromptTemplate

prompt_translate = PromptTemplate.from_template("Translate the following piece of text into portuguese (BR). Provide the translation ONLY. \n {text}")

# model_translate = Ollama(model="llama3.1", temperature = 0.1)

chain_translate = prompt_translate | model | StrOutputParser()

print(chain_translate.invoke({'text': 'Be nice to people all the time, Jeremy'}))

Sê amável com as pessoas o tempo todo, Jeremy.


In [24]:
print(chain.output_schema.schema())

print(chain_translate.input_schema.schema())

{'title': 'StrOutputParserOutput', 'type': 'string'}
{'properties': {'text': {'title': 'Text', 'type': 'string'}}, 'required': ['text'], 'title': 'PromptInput', 'type': 'object'}


# 4 - Compondo pipelines de tradução com a de piadas

In [59]:
from langchain_core.runnables import RunnablePassthrough

composed_chain = chain | {'text': RunnablePassthrough()} | chain_translate

print(composed_chain.invoke({'topic': 'computer'}))

"Por que o computador foi para terapia? Porque ele tinha um vírus... e não só no seu disco rígido, mas também em seu programa de código. Sim, é um problema realmente de bytes!" (bum-tss)


In [51]:
prompt_input = PromptTemplate.from_template("Translate the following portuguese topic into english. Provide ONLY the translation. \n {tema}")

composed_chain = (
    prompt_input 
    | model 
    | StrOutputParser()
    | {'topic': RunnablePassthrough()}
    | chain
    | {'text': RunnablePassthrough()}
    | chain_translate
)

stream = composed_chain.stream({'tema', 'brasileiros'})
for chunk in stream:
    print(chunk, end="", flush=True)
print()

Aqui vamos!

Por que o brasileiro levou uma escada para a festa?

Porque ouviu dizer que as bebidas estavam de graça! (entendeu?)
